In [0]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import time
import os
from PIL import Image
from matplotlib import pyplot
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

img_rows = 120
img_cols = 120
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 120

def build_generator():
    model = Sequential()
    model.add(Dense(512 * 40 * 40, activation="relu", input_dim=latent_dim))
    model.add(Reshape((40, 40, 512)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
    model.summary()
    noise = Input(shape=(latent_dim,))
    img = model(noise)
    return model

def build_discriminator():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    img = Input(shape=img_shape)
    validity = model(img)
    return model

optimizer = Adam(0.0002, 0.5)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

generator = build_generator()
z = Input(shape=(120,))
img = generator(z)

discriminator.trainable = False
valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

def train(epochs, batch_size, save_interval):
    os.makedirs('images', exist_ok=True)
    #/content/drive/My Drive/Colab Notebooks/CSV_file/002aa.csv
    # Load the dataset
    #(X_train, _), (_, _) = mnist.load_data()
    read=pd.read_csv("/content/drive/My Drive/Colab Notebooks/CSV_file/1k.csv").as_matrix()
    arr=np.array
    arr = read.reshape(740,120,120)
    X_train=arr

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random real images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        # Sample noise and generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise)

        # Train the discriminator
        D_loss_real = discriminator.train_on_batch(real_imgs, valid)
        D_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
        D_loss = 0.5 * np.add(D_loss_real, D_loss_fake)

        # Train the generator
        g_loss = combined.train_on_batch(noise, valid)

        # If at save interval
        if epoch % save_interval == 0:
            # Print the progress
            print('>Accuracy real: %.0f%%, fake: %.0f%%' % (D_loss_real[0]*100, D_loss_fake[0]*100))
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, D_loss[0], 100 * D_loss[1], g_loss))
            # Save generated image samples
            save_imgs(epoch)

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    #drive/My Drive/Images/002aa_images/
    fig.savefig("drive/My Drive/Images-120/Images-400batch/1k_images/1k_%d.png" % epoch)
    plt.close()

start = time.time()
train(epochs=10000, batch_size=2, save_interval=400)
end = time.time()
elapsed_train_time = 'elapsed training time: {} min, {} sec '.format(int((end - start) / 60),int((end - start) % 60))
print(elapsed_train_time)

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 80, 80, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 80, 80, 32)     

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>Accuracy real: 70%, fake: 301%
0 [D loss: 1.854880, acc.: 25.00%] [G loss: 0.780317]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>Accuracy real: 141%, fake: 555%
400 [D loss: 3.478141, acc.: 0.00%] [G loss: 0.447883]
>Accuracy real: 24%, fake: 141%
800 [D loss: 0.827452, acc.: 75.00%] [G loss: 4.789582]
>Accuracy real: 11%, fake: 359%
1200 [D loss: 1.852149, acc.: 50.00%] [G loss: 4.038705]
>Accuracy real: 2%, fake: 3%
1600 [D loss: 0.023052, acc.: 100.00%] [G loss: 9.128578]
>Accuracy real: 128%, fake: 12%
2000 [D loss: 0.697339, acc.: 75.00%] [G loss: 0.065807]
>Accuracy real: 105%, fake: 11%
2400 [D loss: 0.580030, acc.: 75.00%] [G loss: 4.358116]
>Accuracy real: 1%, fake: 0%
2800 [D loss: 0.005549, acc.: 100.00%] [G loss: 10.103676]
>Accuracy real: 6%, fake: 0%
3200 [D loss: 0.032545, acc.: 100.00%] [G loss: 7.839793]
>Accuracy real: 0%, fake: 188%
3600 [D loss: 0.941831, acc.: 50.00%] [G loss: 10.646711]
>Accuracy real: 3%, fake: 3%
4000 [D loss: 0.027766, acc.: 100.00%] [G loss: 9.039549]
>Accuracy real: 6%, fake: 0%
4400 [D loss: 0.029296, acc.: 100.00%] [G loss: 15.564871]
>Accuracy real: 0%, fake: 26%
4